In [1]:
spark.stop()

In [2]:
from pyspark.sql import SparkSession
import delta
import pandas as pd

spark = (SparkSession.builder.master("local[*]")
    .appName("TesteHive")
    .config("hive.metastore.uris", "thrift://192.168.15.4:9083")
    .config("spark.sql.warehouse.dir","hdfs://localhost:9000/users/hive/warehouse")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.adaptive.enabled", "true")
    .enableHiveSupport()
    .getOrCreate())
spark

In [12]:
spark.conf.set("SQLConf.ADAPTIVE_EXECUTION_ENABLED.key", "true")

In [6]:
import sys
sys.path.insert(0, "C:/Users/Daniel/Desktop/Estudos/DataSus/notebooks")
from ingestors import IngestaoBronze
import dbtools

In [7]:
table = "rd_sih"
path_full_load = f'/DataSus/rd/csv'
path_incremental = f'/DataSus/rd/csv'
file_format = 'csv'
table_name=table
database_name='bronze'
id_fields = ["N_AIH","DT_SAIDA","IDENT"]
timestamp_field= 'DT_SAIDA'
partition_fields=["ANO_CMPT","MES_CMPT"]
read_options = {'sep': ';','header': 'true'}

ingestao = IngestaoBronze(
            path_full_load=path_full_load,
            path_incremental=path_incremental,
            file_format=file_format,
            table_name=table_name,
            database_name=database_name,
            id_fields=id_fields,
            timestamp_field=timestamp_field,
            partition_fields=partition_fields,
            read_options=read_options,
            spark=spark)

Inferindo schema...
ok
Carregando query default
Ok.


In [8]:
if not spark._jsparkSession.catalog().tableExists(database_name, table):
    df_null = spark.createDataFrame(data=[], schema=ingestao.schema)
    ingestao.save_full(df_null)
    #dbutils.fs.rm(ingestao.checkpoint_path, True) -- Ver como fazer essa parte - acho q vou usar a lib do hdfs

In [9]:
ingestao.process_stream()

AttributeError: 'DataStreamWriter' object has no attribute 'sparkSession'

In [49]:
spark.sql('''

select 

substring(UF_zi,1,2) as UF,
count (distinct 0),
count (distinct mes_cmpt),
count (distinct N_AIH,DT_SAIDA,IDENT),
count(*)

from bronze.rd_sih

group by 1
order by 1
''').show(30)

+---+-----------------+------------------------+--------------------------------------+--------+
| UF|count(DISTINCT 0)|count(DISTINCT mes_cmpt)|count(DISTINCT N_AIH, DT_SAIDA, IDENT)|count(1)|
+---+-----------------+------------------------+--------------------------------------+--------+
| 12|                1|                       6|                                 25169|   25169|
| 16|                1|                       6|                                 20748|   20748|
+---+-----------------+------------------------+--------------------------------------+--------+



In [46]:
df = spark.read.csv('/DataSus/rd/csv',sep=";",header=True)
df.createOrReplaceTempView("test")

In [47]:
spark.sql('''

select 

substring(UF_zi,1,2) as UF,
count (distinct ano_cmpt),
count (distinct mes_cmpt),
count (distinct N_AIH,DT_SAIDA,IDENT),
count(*)

from test

group by 1
order by 1
''').show(30)

+---+------------------------+------------------------+--------------------------------------+--------+
| UF|count(DISTINCT ano_cmpt)|count(DISTINCT mes_cmpt)|count(DISTINCT N_AIH, DT_SAIDA, IDENT)|count(1)|
+---+------------------------+------------------------+--------------------------------------+--------+
| 12|                       1|                       6|                                 25169|   29354|
| 16|                       1|                       6|                                 20748|   24097|
+---+------------------------+------------------------+--------------------------------------+--------+

